### Original model with larger data set

We have used a series of modern "tricks" to improve the performance of the 1989 model, without changing its basic structure.

But one of our best "tricks" for improving the performance of large neural networks is to increase the size and/or quality of the data on which the model is trained.

It is interesting to see what performance we could have achieved in 1989 without any of those other "tricks" (some of which were not yet understood or widely used) - just using more data.


In the following cells, we preprocess the full MNIST dataset - 50,000 training samples and 10,000 test samples, rather than the 7291 and 2007 samples used in 1989 - to downsample them from 28x28 to 16x16.

Then, we'll train that original 1989 model - *without* any of the changes we tried earlier - on this larger data set.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets

torch.set_num_threads(2) # for performance

In [ ]:
# The following code is adapted from "prepro.py" in the repository.

torch.manual_seed(1337)
np.random.seed(1337)

for split in ['test', 'train']:

    data = datasets.MNIST('./data', train=split=='train', download=True)

    # Note: use full size data set
    n = 50000 if split == 'train' else 10000
    rp = np.random.permutation(len(data))[:n]

    X = torch.full((n, 1, 16, 16), 0.0, dtype=torch.float32)
    Y = torch.full((n, 10), -1.0, dtype=torch.float32)
    for i, ix in enumerate(rp):
        I, yint = data[int(ix)]
        # PIL image -> numpy -> torch tensor -> [-1, 1] fp32
        xi = torch.from_numpy(np.array(I, dtype=np.float32)) / 127.5 - 1.0
        # add a fake batch dimension and a channel dimension of 1 or F.interpolate won't be happy
        xi = xi[None, None, ...]
        # resize to (16, 16) images with bilinear interpolation
        xi = F.interpolate(xi, (16, 16), mode='bilinear')
        X[i] = xi[0] # store

        # set the correct class to have target of +1.0
        Y[i, yint] = 1.0

    torch.save((X, Y), split + 'full.pt')

100%|██████████| 9912422/9912422 [00:00<00:00, 71638042.80it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 9457814.95it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 39912114.62it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 9734557.37it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# The following code is adapted from "repro.py" in the repository.

class BaseNet(nn.Module):
    """ 1989 LeCun ConvNet per description in the paper """

    def __init__(self):
        super().__init__()

        # initialization as described in the paper to my best ability, but it doesn't look right...
        winit = lambda fan_in, *shape: (torch.rand(*shape) - 0.5) * 2 * 2.4 / fan_in**0.5
        macs = 0 # keep track of MACs (multiply accumulates)
        acts = 0 # keep track of number of activations

        # H1 layer parameters and their initialization
        self.H1w = nn.Parameter(winit(5*5*1, 12, 1, 5, 5))
        self.H1b = nn.Parameter(torch.zeros(12, 8, 8)) # presumably init to zero for biases
        assert self.H1w.nelement() + self.H1b.nelement() == 1068
        macs += (5*5*1) * (8*8) * 12
        acts += (8*8) * 12

        # H2 layer parameters and their initialization
        """
        H2 neurons all connect to only 8 of the 12 input planes, with an unspecified pattern
        I am going to assume the most sensible block pattern where 4 planes at a time connect
        to differently overlapping groups of 8/12 input planes. We will implement this with 3
        separate convolutions that we concatenate the results of.
        """
        self.H2w = nn.Parameter(winit(5*5*8, 12, 8, 5, 5))
        self.H2b = nn.Parameter(torch.zeros(12, 4, 4)) # presumably init to zero for biases
        assert self.H2w.nelement() + self.H2b.nelement() == 2592
        macs += (5*5*8) * (4*4) * 12
        acts += (4*4) * 12

        # H3 is a fully connected layer
        self.H3w = nn.Parameter(winit(4*4*12, 4*4*12, 30))
        self.H3b = nn.Parameter(torch.zeros(30))
        assert self.H3w.nelement() + self.H3b.nelement() == 5790
        macs += (4*4*12) * 30
        acts += 30

        # output layer is also fully connected layer
        self.outw = nn.Parameter(winit(30, 30, 10))
        self.outb = nn.Parameter(-torch.ones(10)) # 9/10 targets are -1, so makes sense to init slightly towards it
        assert self.outw.nelement() + self.outb.nelement() == 310
        macs += 30 * 10
        acts += 10

        self.macs = macs
        self.acts = acts

    def forward(self, x):

        # x has shape (1, 1, 16, 16)
        x = F.pad(x, (2, 2, 2, 2), 'constant', -1.0) # pad by two using constant -1 for background
        x = F.conv2d(x, self.H1w, stride=2) + self.H1b
        x = torch.tanh(x)

        # x is now shape (1, 12, 8, 8)
        x = F.pad(x, (2, 2, 2, 2), 'constant', -1.0) # pad by two using constant -1 for background
        slice1 = F.conv2d(x[:, 0:8], self.H2w[0:4], stride=2) # first 4 planes look at first 8 input planes
        slice2 = F.conv2d(x[:, 4:12], self.H2w[4:8], stride=2) # next 4 planes look at last 8 input planes
        slice3 = F.conv2d(torch.cat((x[:, 0:4], x[:, 8:12]), dim=1), self.H2w[8:12], stride=2) # last 4 planes are cross
        x = torch.cat((slice1, slice2, slice3), dim=1) + self.H2b
        x = torch.tanh(x)

        # x is now shape (1, 12, 4, 4)
        x = x.flatten(start_dim=1) # (1, 12*4*4)
        x = x @ self.H3w + self.H3b
        x = torch.tanh(x)

        # x is now shape (1, 30)
        x = x @ self.outw + self.outb
        x = torch.tanh(x)

         # x is finally shape (1, 10)
        return x

In [ ]:
learning_rate = 0.03

# init rng
torch.manual_seed(1337)
np.random.seed(1337)
torch.use_deterministic_algorithms(True)

# init a model
model = BaseNet()
print("model stats:")
print("# params:      ", sum(p.numel() for p in model.parameters())) # in paper total is 9,760
print("# MACs:        ", model.macs)
print("# activations: ", model.acts)

# init data
Xtr, Ytr = torch.load('trainfull.pt')
Xte, Yte = torch.load('testfull.pt')

# init optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

def eval_split(split):
    # eval the full train/test set, batched implementation for efficiency
    model.eval()
    X, Y = (Xtr, Ytr) if split == 'train' else (Xte, Yte)
    Yhat = model(X)
    loss = torch.mean((Y - Yhat)**2)
    err = torch.mean((Y.argmax(dim=1) != Yhat.argmax(dim=1)).float())
    print(f"eval: split {split:5s}. loss {loss.item():e}. error {err.item()*100:.2f}%. misses: {int(err.item()*Y.size(0))}")

# train
for pass_num in range(80):

    # perform one epoch of training
    model.train()
    for step_num in range(Xtr.size(0)):

        # fetch a single example into a batch of 1
        x, y = Xtr[[step_num]], Ytr[[step_num]]

        # forward the model and the loss
        yhat = model(x)
        loss = torch.mean((y - yhat)**2)

        # calculate the gradient and update the parameters
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # after epoch epoch evaluate the train and test error / metrics
    print(pass_num + 1)
    eval_split('train')
    eval_split('test')

# save final model to file
torch.save(model.state_dict(), 'base_model_data.pt')

model stats:
# params:       9760
# MACs:         63660
# activations:  1000
1
eval: split train. loss 3.372094e-02. error 5.11%. misses: 2555
eval: split test . loss 3.295155e-02. error 5.04%. misses: 504
2
eval: split train. loss 2.436154e-02. error 3.58%. misses: 1790
eval: split test . loss 2.541548e-02. error 3.73%. misses: 373
3
eval: split train. loss 2.174933e-02. error 3.24%. misses: 1620
eval: split test . loss 2.369628e-02. error 3.57%. misses: 357
4
eval: split train. loss 2.092772e-02. error 3.16%. misses: 1579
eval: split test . loss 2.357459e-02. error 3.49%. misses: 348
5
eval: split train. loss 1.849931e-02. error 2.74%. misses: 1369
eval: split test . loss 2.216178e-02. error 3.43%. misses: 342
6
eval: split train. loss 1.842882e-02. error 2.75%. misses: 1372
eval: split test . loss 2.217882e-02. error 3.43%. misses: 342
7
eval: split train. loss 1.781441e-02. error 2.66%. misses: 1329
eval: split test . loss 2.282108e-02. error 3.53%. misses: 353
8
eval: split train.

Using the original 1989 model, 

* with the "weird" MSE loss function, 
* tanh activation, 
* "plain" stochastic gradient descent with a fixed learning rate, 
* and no regularization, 

we still see a very respectable increase in performance just by training on more data (and since there is more data, adding more training passes.)

This would have substantially increased the training time in 1989 - the original model trained on 7291 training samples took 3 days to train, and this training set is about 7x that size - but the inference time would not have been affected.

### Modern model with larger data set

Finally, let's see how our "modern" updated neural network does when trained on the larger dataset - 50,000 training samples.

In [ ]:
class ModernNet(nn.Module):

    def __init__(self):
        super().__init__()

        # initialization as described in the paper to my best ability, but it doesn't look right...
        winit = lambda fan_in, *shape: (torch.rand(*shape) - 0.5) * 2 * 2.4 / fan_in**0.5
        macs = 0 # keep track of MACs (multiply accumulates)
        acts = 0 # keep track of number of activations

        # H1 layer parameters and their initialization
        self.H1w = nn.Parameter(winit(5*5*1, 12, 1, 5, 5))
        self.H1b = nn.Parameter(torch.zeros(12, 8, 8)) # presumably init to zero for biases
        macs += (5*5*1) * (8*8) * 12
        acts += (8*8) * 12

        # H2 layer parameters and their initialization
        """
        H2 neurons all connect to only 8 of the 12 input planes, with an unspecified pattern
        I am going to assume the most sensible block pattern where 4 planes at a time connect
        to differently overlapping groups of 8/12 input planes. We will implement this with 3
        separate convolutions that we concatenate the results of.
        """
        self.H2w = nn.Parameter(winit(5*5*8, 12, 8, 5, 5))
        self.H2b = nn.Parameter(torch.zeros(12, 4, 4)) # presumably init to zero for biases
        macs += (5*5*8) * (4*4) * 12
        acts += (4*4) * 12

        # H3 is a fully connected layer
        self.H3w = nn.Parameter(winit(4*4*12, 4*4*12, 30))
        self.H3b = nn.Parameter(torch.zeros(30))
        macs += (4*4*12) * 30
        acts += 30

        # output layer is also fully connected layer
        self.outw = nn.Parameter(winit(30, 30, 10))
        self.outb = nn.Parameter(torch.zeros(10))
        macs += 30 * 10
        acts += 10

        self.macs = macs
        self.acts = acts

    def forward(self, x):

        # poor man's data augmentation by 1 pixel along x/y directions
        if self.training:
            shift_x, shift_y = np.random.randint(-1, 2, size=2)
            x = torch.roll(x, (shift_x, shift_y), (2, 3))

        # x has shape (1, 1, 16, 16)
        x = F.pad(x, (2, 2, 2, 2), 'constant', -1.0) # pad by two using constant -1 for background
        x = F.conv2d(x, self.H1w, stride=2) + self.H1b
        x = torch.relu(x) # Note: ReLU activation!

        # x is now shape (1, 12, 8, 8)
        x = F.pad(x, (2, 2, 2, 2), 'constant', -1.0) # pad by two using constant -1 for background
        slice1 = F.conv2d(x[:, 0:8], self.H2w[0:4], stride=2) # first 4 planes look at first 8 input planes
        slice2 = F.conv2d(x[:, 4:12], self.H2w[4:8], stride=2) # next 4 planes look at last 8 input planes
        slice3 = F.conv2d(torch.cat((x[:, 0:4], x[:, 8:12]), dim=1), self.H2w[8:12], stride=2) # last 4 planes are cross
        x = torch.cat((slice1, slice2, slice3), dim=1) + self.H2b
        x = torch.relu(x) # Note: ReLU activation!

        # x is now shape (1, 12, 4, 4)
        x = x.flatten(start_dim=1) # (1, 12*4*4)
        x = x @ self.H3w + self.H3b
        x = torch.relu(x) # Note: ReLU activation!
        x = F.dropout(x, p=0.25, training=self.training) # Note: dropout layer!


        # x is now shape (1, 30)
        x = x @ self.outw + self.outb

         # x is finally shape (1, 10)
        return x

In [ ]:
# The following code is adapted from "modern.py" in the repository.

learning_rate = 3e-4

# init rng
torch.manual_seed(1337)
np.random.seed(1337)
torch.use_deterministic_algorithms(True)

# init a model
model = ModernNet()
print("model stats:")
print("# params:      ", sum(p.numel() for p in model.parameters())) # in paper total is 9,760
print("# MACs:        ", model.macs)
print("# activations: ", model.acts)

# init data
Xtr, Ytr = torch.load('trainfull.pt')
Xte, Yte = torch.load('testfull.pt')

# init optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

def eval_split(split):
    # eval the full train/test set, batched implementation for efficiency
    model.eval()
    X, Y = (Xtr, Ytr) if split == 'train' else (Xte, Yte)
    Yhat = model(X)
    loss = F.cross_entropy(Yhat, Y.argmax(dim=1))
    err = torch.mean((Y.argmax(dim=1) != Yhat.argmax(dim=1)).float())
    print(f"eval: split {split:5s}. loss {loss.item():e}. error {err.item()*100:.2f}%. misses: {int(err.item()*Y.size(0))}")
  
# train
for pass_num in range(80):

    # learning rate decay
    alpha = pass_num / 79
    for g in optimizer.param_groups:
        g['lr'] = (1 - alpha) * learning_rate + alpha * (learning_rate / 3)

    # perform one epoch of training
    model.train()
    for step_num in range(Xtr.size(0)):

        # fetch a single example into a batch of 1
        x, y = Xtr[[step_num]], Ytr[[step_num]]

        # forward the model and the loss
        yhat = model(x)
        loss = F.cross_entropy(yhat, y.argmax(dim=1))

        # calculate the gradient and update the parameters
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # after epoch epoch evaluate the train and test error / metrics
    print(pass_num + 1)
    eval_split('train')
    eval_split('test')

# save final model to file
torch.save(model.state_dict(), 'modern_model_data.pt')

model stats:
# params:       9760
# MACs:         63660
# activations:  1000
1
eval: split train. loss 1.800405e-01. error 5.34%. misses: 2669
eval: split test . loss 1.658235e-01. error 4.76%. misses: 476
2
eval: split train. loss 1.235864e-01. error 3.79%. misses: 1893
eval: split test . loss 1.099108e-01. error 3.30%. misses: 329
3
eval: split train. loss 1.132230e-01. error 3.56%. misses: 1779
eval: split test . loss 9.745394e-02. error 3.22%. misses: 322
4
eval: split train. loss 9.968607e-02. error 2.95%. misses: 1475
eval: split test . loss 9.140631e-02. error 2.69%. misses: 269
5
eval: split train. loss 1.013773e-01. error 2.98%. misses: 1489
eval: split test . loss 8.744171e-02. error 2.56%. misses: 255
6
eval: split train. loss 9.772331e-02. error 2.87%. misses: 1434
eval: split test . loss 8.561321e-02. error 2.71%. misses: 271
7
eval: split train. loss 9.237381e-02. error 2.82%. misses: 1408
eval: split test . loss 8.389988e-02. error 2.62%. misses: 262
8
eval: split train.

With the extra data, our "updated" version of the 1989 model, with

* MSE loss changed to cross entropy loss 
* tanh activation changed to ReLU
* SGD with constant learning rate changed to Adam with learning rate decay
* added dropout layer for regularization
* basic data augmentation (shift up to 1 pixel in each direction)

but the same basic structure, easily achieves less than 2% error.

## Summary of results

To summarize, the test error rate of each model was:

* Original 1989 paper: **5.0%**
* Pytorch model based on original: **4.14%**
* ... + cross entropy loss: **4.58%**
* ... + Adam optimizer: **3.89%**
* ... + data augmentation: **2.29%**
* ... + dropout + ReLU activations: **2.09%**
* ... + more training data: **1.45%**

and, 

* Pytorch model based on 1989 original + more training data: about **3%**